#### Vamos a entrenar un detector de personas usando yolov5. Para ello, a partir del dataset de coco. En primer lugar extraemos las imágenes y las anotaciones de los bounding boxes del dataset de Coco para la clase persona

#### Usamos el repositorio de github de yolov5 para realizar la detección

##### Debemos configurar un entrono virtual de trabajo. 
##### Creamos un entorno conda create -n entonoYolo python = 3.8
##### Activamos el entorno conda activate entornoYolo
##### Nos desplazamos a la carpeta de trabajo
##### Clonamos el entorno de trabajo de yolov5: https://github.com/ultralytics/yolov5.git 

##### Nos desplazamos a la carpeta de yolov5: cd yolov5
##### Instalamos las dependencias necesarias para instalar yolov5: pip install - requirements.txt 
##### Falta la librería typeguard (da error al ejecutar el comando anterior) pip install typeguard
##### Instalamos jupyter-notebooks: pip install jupyter 

##### Con esta configuración estamos listos ya para usar yolov5 

#### Importamos las bibliotecas que sean necesarias

In [1]:
from pycocotools.coco import COCO #Nos va a servir para cargar las imágenes y las anotaciones de bounding box para la clase persona
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
import torch

import shutil

#### Hacemos git clone del repositorio de github de yolov5

In [2]:
#!git clone --recursive https://github.com/ultralytics/yolov5.git

##### Se comprueba que la tarjeta gráfica está disponible en el entorno para la realización del entrenamiento.

In [3]:
print(f"Carga completada. Estamos usando torch version {torch.__version__}")
print(f"GPU disponible: {torch.cuda.is_available()}")

Carga completada. Estamos usando torch version 2.3.0+cu121
GPU disponible: True


#### Creamos los directorios necesarios para almacenar las imágenes y las anotaciones para yolo.

In [4]:
ruta_actual = os.getcwd()
ruta_yolov5 = os.path.join(ruta_actual,"yolov5")
ruta_datos_entrenamiento_validacion = os.path.join(ruta_yolov5,"datos_deteccion_personas")

#Creamos la carpeta datos_deteccion_caras dentro de la carpeta de yolov5 (se obtuvo al hacer el git clone del repositorio de github)
try:
  os.mkdir(ruta_datos_entrenamiento_validacion)
except:
  print("EL directorio ya existe")

In [5]:
ruta_imagenes = os.path.join(ruta_datos_entrenamiento_validacion,"images")
try:
  os.mkdir(ruta_imagenes)
except:
  print("El directorio imágenes ya existe")

ruta_imagenes_entrenamiento = os.path.join(ruta_imagenes,"train") 
try:
  os.mkdir(ruta_imagenes_entrenamiento)
except:
  print("El directorio de imágenes de entrenamiento ya existe")

ruta_imagenes_validacion = os.path.join(ruta_imagenes,"val")
try:
  os.mkdir(ruta_imagenes_validacion)
except:
  print("El directorio de imágenes validacion ya existe")


ruta_etiquetas = os.path.join(ruta_datos_entrenamiento_validacion,"labels")
try:
  os.mkdir(ruta_etiquetas)
except:
  print("El directorio etiquetas ya existe")

ruta_etiquetas_entrenamiento = os.path.join(ruta_etiquetas,"train")
try:
  os.mkdir(ruta_etiquetas_entrenamiento)
except:
  print("El directorio etiquetas de entrenamiento ya existe")

ruta_etiquetas_validacion = os.path.join(ruta_etiquetas,"val")
try:
  os.mkdir(ruta_etiquetas_validacion)
except:
  print("El directorio etiquetas de validación ya existe")

#### Creamos una función para transformar los datos de las bounding boxes a formato Yolo.

In [6]:
#Creamos una función que pasa las anotaciones de wider dataset al formato de yolo
def caja_yolo(x_esquina_superior,y_esquina_superior,ancho_bb,alto_bb,ruta_imagen):

    #no hace falta realizar el casting
    #x_esquina_superior = int(x_esquina_superior)
    #y_esquina_superior = int(y_esquina_superior)
    #ancho_bb = int(ancho_bb)
    #alto_bb = int(alto_bb)
        
    imagen = cv2.imread(ruta_imagen)
    filas,columnas,_ = imagen.shape
    x = (x_esquina_superior + ancho_bb/2)/columnas
    y = (y_esquina_superior + alto_bb/2)/filas 
    w = ancho_bb/columnas
    h = alto_bb/filas
    return x,y,w,h

#### Usamos la API de coco para seleccionar las imágenes de personas y las anotaciones para las bounding boxes.
#### Creamos una función que realiza el proceso

In [7]:
# Ruta a los archivo de anotaciones de Coco
ruta_anotaciones_entrenamiento = '/media/roger/Datos/asignaturas_master/tfm/coco_dataset/annotations_trainval2017/annotations/instances_train2017.json'
ruta_anotaciones_validacion = '/media/roger/Datos/asignaturas_master/tfm/coco_dataset/annotations_trainval2017/annotations/instances_val2017.json'

numero_anotaciones_entrenamiento = 0
numero_anotaciones_validacion = 0



def crear_anotaciones(ruta_anotaciones):
    global numero_anotaciones_entrenamiento
    global numero_anotaciones_validacion

    # Cargar anotaciones de coco
    anotaciones_coco = COCO(ruta_anotaciones)

    #Obtenemos la categoría ID para la clase personas
    categoria_persona = anotaciones_coco.getCatIds(catNms=['person'])

    print(categoria_persona)

    #El id para la categoría persona es el número 1. Obtenemos todas las imágenes en las que aparecen personas, así como las bounding boxes.
    # Obtenemos todas los ids/números de imagenes relativos a personas
    id_imagenes_personas = anotaciones_coco.getImgIds(catIds=categoria_persona)
    #obtenemos una lista de números naturales que hacen referencias a personas.
    #Las imágenes tienen el nombre 000000numero.jpg donde número es alguno de los números de la lista (12 dígitos en total)


    info_general_imagenes_personas = anotaciones_coco.loadImgs(id_imagenes_personas)

    # Guardamos las imágenes y los txt con las anotaciones en la estructura de carpetas de yolo


    for info_imagen in info_general_imagenes_personas:

    
        ruta_dataset_coco = "/media/roger/Datos/asignaturas_master/tfm/coco_dataset"

        nombre_imagen = info_imagen['file_name']
        url = info_imagen['coco_url']
        ruta_relativa = os.path.join(*url.split('/')[-2:])
        tipo_entrenamiento_validacion = os.path.join(*url.split('/')[-2:-1])
        #ancho_imagen = info_imagen['width']
        #alto_imagen = info_imagen['height']
        ruta_imagen = os.path.join(ruta_dataset_coco,ruta_relativa)

        #Guardamos la imagen en el directorio adecuado. Creamos el txt en el directorio adecuado
        if tipo_entrenamiento_validacion == "train2017":
            #Copiamos imagen de entrenamiento al directorio adecuado de yolo
            shutil.copy(ruta_imagen,ruta_imagenes_entrenamiento)
            #Creamos el txt con las anotaciones de entrenamiento en el directorio adecuado de yolo
            nombre_txt = os.path.basename(nombre_imagen.strip())[:-3] + 'txt'
            txt = os.path.join(ruta_etiquetas_entrenamiento, nombre_txt)
    
        elif tipo_entrenamiento_validacion == "val2017":
            #Copiamos la imagen de validación al directorio adecuado de yolo
            shutil.copy(ruta_imagen,ruta_imagenes_validacion)
            #Creamos el txt con las anotaciones de validación en el directorio adecuado de yolo
            nombre_txt = os.path.basename(nombre_imagen.strip())[:-3] + 'txt'
            txt = os.path.join(ruta_etiquetas_validacion, nombre_txt)
        
        else:
            print("Hubo un error. La imagen no pertenece ni a train ni a val")
    
        #Obtenemos las anotaciones para las imágenes. En particular sólo tomaremos las bounding boxes ya que sólo esos datos nos interesan
        imagen_id = info_imagen['id']    

        anotaciones_ids = anotaciones_coco.getAnnIds(imgIds=imagen_id, catIds=categoria_persona, iscrowd=None)
        anotaciones_personas_imagen = anotaciones_coco.loadAnns(anotaciones_ids)    

        #recorremos con el bucle for todas las anotaciones y tomamos las relativas a los bounding box
        #Grabamos las anotaciones en txt en los directorios adecuados

        #Primero abrimos el txt
        archivo_txt = open(txt,"a")
        for anotacion in anotaciones_personas_imagen:
        
            #Incrementamos en uno el contador de anotaciones
            if tipo_entrenamiento_validacion == "train2017":
                numero_anotaciones_entrenamiento += 1
            elif tipo_entrenamiento_validacion == "val2017":
                numero_anotaciones_validacion += 1
            else:
                print("Hubo un error. La imagen no pertenece ni a train ni a val")
        
            bounding_box = anotacion['bbox']
            x,y,w,h = bounding_box

            #Procesamos los bounding box y los transformamos al formato yolo
            x,y,w,h = caja_yolo(x,y,w,h,ruta_imagen)
        
            #Grabamos las anotaciones en formato yolo en el txt
            archivo_txt.write(str(0) + " ")
            archivo_txt.write(str(x) + " ")
            archivo_txt.write(str(y) + " ")
            archivo_txt.write(str(w) + " ")
            archivo_txt.write(str(h))
            archivo_txt.write("\n")
    
        #Una vez grabadas las anotaciones, cerramos el txt
        archivo_txt.close()


#### Generamos anotaciones e imágenes de entrenamiento

In [8]:
crear_anotaciones(ruta_anotaciones_entrenamiento)

loading annotations into memory...
Done (t=6.75s)
creating index...
index created!
[1]


#### Generamos anotaciones e imágenes de validación

In [9]:
crear_anotaciones(ruta_anotaciones_validacion)

loading annotations into memory...
Done (t=0.20s)
creating index...
index created!
[1]


#### Mostramos la información de imágenes y las anotaciones en entrenamiento y test

In [10]:
print(f"El número de imágenes de entrenamiento es {len(os.listdir(ruta_imagenes_entrenamiento))} con un total de {numero_anotaciones_entrenamiento} anotaciones")

print(f"El número de imágenes de validación es {len(os.listdir(ruta_imagenes_validacion))} con un total de {numero_anotaciones_validacion} anotaciones")

El número de imágenes de entrenamiento es 64115 con un total de 262465 anotaciones
El número de imágenes de validación es 2693 con un total de 11004 anotaciones


#### Entrenamos el modelo durante 75 épocas usando el script train.py del repositorio de github de yolov5. En mi caso usé la opción --cache para acelerar el entrenamiento (esto sólo debe hacerse si se dispone de una memoria ram grande que sea capaz de almacenar todas las imágenes de entrenamiento y validación en memoria ram). El batch se tomo de 8 (es adecuado para mi GPU con 8 GB de VRAM)

#### En la carpeta  se almacena información del entrenamiento y datos sobre el conjunto de validación. En la carpeta weights dentro del directorio yolov5/runs/train se almacenan los pesos del último modelo y del mejor modelo resultado del entrenamiento. en results.csv se almacenan datos de cada época de entrenamiento. Aparecen también gráficas de entrenamiento (matrtiz de confusión, curva F1, curva precision/recall, etc)

In [15]:
#Probamos con un modelo intermedio yolov5m (detectar personas es más fácil que detectar caras). Puede verse en https://github.com/ultralytics/yolov5
!python ./yolov5/train.py --batch 16 --epochs 50 --patience 10 --data ./yolov5/data/custom.yaml --weights yolov5m.pt --cache

#Probamos con un tamaño de lote grande (32) (32 es demasiado batch-size) porque el modelo es más ligero que otros (21  millones de parámetros)
#Usar un modelo más pequeño puede beneficiar el entrenamiento ya que tenemos muchas imágenes etiquetadas 
#No pude meter las imágenes en cache porque son demasiadas. Intentamos entrenar 50 época.
#Puede que considere la posibilidad de reducir el tamaño del dataset.

train: weights=yolov5m.pt, cfg=, data=./yolov5/data/custom.yaml, hyp=yolov5/data/hyps/hyp.scratch-low.yaml, epochs=50, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=yolov5/data/hyps, resume_evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=yolov5/runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=10, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest, ndjson_console=False, ndjson_file=False
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v7.0-353-g5eca7b9c Python-3.10.13 torch-2.3.0+cu121 CUDA:0 (NVIDIA GeForce RTX 4070 Laptop GPU, 7940MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.

#### Los resultados obtenidos entrenando 50 épocas son bastante buenos

#### Cargamos el modelo de detección con el mejor de los pesos

In [17]:
modelo = torch.hub.load('ultralytics/yolov5','custom',path='./yolov5/runs/train/exp/weights/best.pt')

Using cache found in /home/roger/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-3-31 Python-3.10.13 torch-2.3.0+cu121 CUDA:0 (NVIDIA GeForce RTX 4070 Laptop GPU, 7940MiB)

Fusing layers... 
Model summary: 212 layers, 20852934 parameters, 0 gradients, 47.9 GFLOPs
Adding AutoShape... 


#### Definimos funciones para la detección 

In [18]:
import subprocess

def deteccion_imagen(ruta):
    imagen = cv2.imread(ruta)
    imagen_rgb = cv2.cvtColor(imagen, cv2.COLOR_BGR2RGB)

    detect=model(imagen_rgb)
    foto = np.squeeze(detect.render())

    plt.imshow(foto)
    plt.show()


#detección de video completo usando el script detect.py del repositorio de github

def deteccion_video(ruta_video, umbral_confianza = 0.25,umbral_iou = 0.45):
    comando = [
        "python", "./yolov5/detect.py",
        "--weights", "./yolov5/runs/train/exp/weights/best.pt",
        "--img", "640",
        "--conf", str(umbral_confianza),
        "--iou-thres", str(umbral_iou),
        "--source", ruta_video
    ]
    
    # Ejecutar el comando
    subprocess.run(comando)
    
    #!python ./yolov5/detect.py --weights './yolov5/runs/train/exp/weights/best.pt' --img 640 --conf 0.25 --source ruta


#### Hacemos la detección sobre dos vídeo . Los resultados son buenos

In [19]:
ruta_video_1 = os.path.join(ruta_actual,"surfistas.mp4")
ruta_video_2 = os.path.join(ruta_actual,"familia.mp4")

#detección en el vídeo de los surfistas
deteccion_video(ruta_video_1, umbral_confianza = 0.25,umbral_iou = 0.45)

#detección en el vídeo de la pareja con niño
deteccion_video(ruta_video_2, umbral_confianza = 0.25,umbral_iou = 0.45)

#Los vídeos con las detecciones se almacenan en ./yolov5/runs/detect
#Se observa que los resultados son buenos

detect: weights=['./yolov5/runs/train/exp/weights/best.pt'], source=/media/roger/Datos/asignaturas_master/tfm/detector_personas/surfistas.mp4, data=yolov5/data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5/runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-353-g5eca7b9c Python-3.10.13 torch-2.3.0+cu121 CUDA:0 (NVIDIA GeForce RTX 4070 Laptop GPU, 7940MiB)

Fusing layers... 
Model summary: 212 layers, 20852934 parameters, 0 gradients, 47.9 GFLOPs
video 1/1 (1/199) /media/roger/Datos/asignaturas_master/tfm/detector_personas/surfistas.mp4: 384x640 2 personas, 44.0ms
video 1/1 (2/199) /media/roger/Datos/asignaturas_master/tfm/detector_personas/surfistas.mp4: 384x640 2 per